In [1]:
#pennylane op-t-mize
import pennylane as qml
import pyzx as zx
import tempfile
import random
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumCircuit, ParameterVector
import csv
import time
import japanize_matplotlib # 追加

In [2]:
#回路セット読み込み
[ds] = qml.data.load("op-t-mize")

pyzx_circuits = []

for i, qscript in enumerate(ds.circuits):
    try:
        # QuantumScript を OpenQASM 2.0 に変換
        qasm_str = qscript.to_openqasm(
            wires=None,
            rotations=False,
            measure_all=False,
            precision=8
        )

        # QASM を一時ファイル経由で PyZX に読み込む
        with tempfile.NamedTemporaryFile(suffix=".qasm", mode='w+', delete=False) as tmpfile:
            tmpfile.write(qasm_str)
            tmpfile.flush()
            zx_circuit = zx.Circuit.load(tmpfile.name)

        # 元の回路名を設定（PyZX Circuit に名前属性を追加）
        zx_circuit.name = ds.circuit_names[i]
        pyzx_circuits.append(zx_circuit)
        
    except Exception as e:
        print(f"回路 {i}（{ds.circuit_names[i]}）の変換に失敗: {e}")

#読み込んだ回路の確認
print(len(pyzx_circuits))
for i, zx_circuit in enumerate(pyzx_circuits):
    dict = zx_circuit.stats_dict()
    print(dict)

31
{'name': 'mod5_4', 'qubits': 5, 'gates': 63, 'tcount': 28, 'clifford': 35, 'twoqubit': 28, 'cnot': 28, 'had': 6, 'measurement': 0, 'other': 0, 'depth': 0, 'depth_cz': 0}
{'name': 'vbe_adder_3', 'qubits': 10, 'gates': 150, 'tcount': 70, 'clifford': 80, 'twoqubit': 70, 'cnot': 70, 'had': 10, 'measurement': 0, 'other': 0, 'depth': 0, 'depth_cz': 0}
{'name': 'csla_mux_3_original', 'qubits': 15, 'gates': 170, 'tcount': 70, 'clifford': 100, 'twoqubit': 80, 'cnot': 80, 'had': 20, 'measurement': 0, 'other': 0, 'depth': 0, 'depth_cz': 0}
{'name': 'csum_mux_9_corrected', 'qubits': 30, 'gates': 448, 'tcount': 196, 'clifford': 252, 'twoqubit': 168, 'cnot': 168, 'had': 56, 'measurement': 0, 'other': 0, 'depth': 0, 'depth_cz': 0}
{'name': 'qcla_com_7', 'qubits': 24, 'gates': 443, 'tcount': 203, 'clifford': 240, 'twoqubit': 186, 'cnot': 186, 'had': 39, 'measurement': 0, 'other': 0, 'depth': 0, 'depth_cz': 0}
{'name': 'qcla_mod_7', 'qubits': 26, 'gates': 884, 'tcount': 413, 'clifford': 471, 'twoqub

In [3]:
pyzx_circuits

[Circuit(5 qubits, 0 bits, 63 gates),
 Circuit(10 qubits, 0 bits, 150 gates),
 Circuit(15 qubits, 0 bits, 170 gates),
 Circuit(30 qubits, 0 bits, 448 gates),
 Circuit(24 qubits, 0 bits, 443 gates),
 Circuit(26 qubits, 0 bits, 884 gates),
 Circuit(36 qubits, 0 bits, 521 gates),
 Circuit(24 qubits, 0 bits, 900 gates),
 Circuit(14 qubits, 0 bits, 200 gates),
 Circuit(11 qubits, 0 bits, 278 gates),
 Circuit(9 qubits, 0 bits, 119 gates),
 Circuit(5 qubits, 0 bits, 45 gates),
 Circuit(7 qubits, 0 bits, 75 gates),
 Circuit(9 qubits, 0 bits, 105 gates),
 Circuit(19 qubits, 0 bits, 255 gates),
 Circuit(12 qubits, 0 bits, 243 gates),
 Circuit(15 qubits, 0 bits, 379 gates),
 Circuit(18 qubits, 0 bits, 545 gates),
 Circuit(21 qubits, 0 bits, 741 gates),
 Circuit(24 qubits, 0 bits, 981 gates),
 Circuit(27 qubits, 0 bits, 1223 gates),
 Circuit(30 qubits, 0 bits, 1509 gates),
 Circuit(48 qubits, 0 bits, 3885 gates),
 Circuit(5 qubits, 0 bits, 60 gates),
 Circuit(7 qubits, 0 bits, 114 gates),
 Circuit

In [ ]:

for i, zx_circuit in enumerate(pyzx_circuits):
    if i>=3:
        break
    c = zx_circuit.copy()

    print(f"before{c.stats()}")
    zx.settings.topt_command = ['/home/rest/forpyzx/TOpt/bin/TOpt']
    c = zx.optimize.phase_block_optimize(c)

    print(f"after {c.stats()}")
    print("--------------")


beforeCircuit mod5_4 on 5 qubits with 63 gates.
        28 is the T-count
        35 Cliffords among which
        28 2-qubit gates (28 CNOT, 0 other) and
        6 Hadamard gates.


TypeError: unsupported operand type(s) for +: 'int' and 'str'